In [2]:
import torch.nn as nn
import torch
import torch
import torch.nn as nn
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from torch.optim.lr_scheduler import LambdaLR
import os
import random
import time
import csv


class Net11(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc_in = nn.Linear(4, 256)

        # blocco 1
        self.b1_1 = nn.Linear(256, 256)
        self.b1_2 = nn.Linear(256, 256)   # skip: in + out

        # blocco 2
        self.b2_1 = nn.Linear(256, 192)
        self.b2_2 = nn.Linear(192, 192)   # skip

        # blocco 3
        self.b3_1 = nn.Linear(192, 128)
        self.b3_2 = nn.Linear(128, 128)   # skip

        # testa finale
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 64)
        self.out = nn.Linear(64, 1)

        self.act = nn.Tanh()

    def forward(self, xmu):
        x = self.act(self.fc_in(xmu))       # 4 → 256

        # ---- blocco 1 ------------------------------------------------
        h = self.act(self.b1_1(x))
        h = self.act(self.b1_2(h))
        x = x + h                           # skip (256)

        # ---- blocco 2 ------------------------------------------------
        h = self.act(self.b2_1(x))          # 256 → 192
        h = self.act(self.b2_2(h))
        x = h + nn.functional.pad(x, (0, -64))  # align dims 256→192

        # ---- blocco 3 ------------------------------------------------
        h = self.act(self.b3_1(x))          # 192 → 128
        h = self.act(self.b3_2(h))
        x = h + x[:, :128]                  # skip (taglia a 128)

        # ---- testa finale -------------------------------------------
        x = self.act(self.fc4(x))
        x = self.act(self.fc5(x))
        return self.out(x)

In [3]:
def set_seed(seed=23):
    random.seed(seed)                        # seed Python random
    np.random.seed(seed)                     # seed numpy random
    torch.manual_seed(seed)                  # seed CPU
    torch.cuda.manual_seed(seed)             # seed GPU (if available)
    torch.cuda.manual_seed_all(seed)         # se hai più GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [4]:
seed = 23
set_seed(seed)

In [8]:
# ------------------------ Calcolo del residuo PDE ------------------------
def pde_residual(xmu, net):
    """
    Calcola il residuo della PDE in ogni punto (interno al dominio)
    -xmu: tensor Nx4 con (x0, x1, mu0, mu1)
    -net: la rete neurale che approssima u
    """
    xmu.requires_grad_(True)      # necessario per calcolare derivate rispetto a xmu
    u = net(xmu)                 # output della rete: u(x,mu)

    # Calcolo gradiente di u rispetto a input (x0, x1) per le derivate prime
    grads = torch.autograd.grad(u.sum(), xmu, create_graph=True)[0]
    u_x0 = grads[:, 0:1]          # ∂u/∂x0
    u_x1 = grads[:, 1:2]          # ∂u/∂x1

    # Calcolo derivate seconde per laplaciano
    u_x0x0 = torch.autograd.grad(u_x0.sum(), xmu, create_graph=True)[0][:, 0:1]  # ∂²u/∂x0²
    u_x1x1 = torch.autograd.grad(u_x1.sum(), xmu, create_graph=True)[0][:, 1:2]  # ∂²u/∂x1²
    laplacian_u = u_x0x0 + u_x1x1  # ∆u

    # Estrazione parametri mu0 e mu1
    mu0 = xmu[:, 2:3]
    mu1 = xmu[:, 3:4]

    # Coordinate spaziali
    x0 = xmu[:, 0:1]
    x1 = xmu[:, 1:2]

    # Termine sorgente g(x;mu) NUOVO EPER 2
    g = 100 * torch.sin(2 * np.pi *mu0* x0) * torch.cos(2 * np.pi *mu0* x1)

    # Termine non lineare con mu0, mu1
    nonlinear = (mu0 / mu1) * (torch.exp(mu1 * u) - 1)

    # Residuo PDE: -∆u + nonlinear - g = 0
    residual = -laplacian_u + nonlinear - g
    return residual

In [9]:
# ------------------------ Generazione punti dominio e bordo ------------------------
def generate_domain_points(N_interior, N_boundary, mu0_range, mu1_range):
    """
    Genera punti interni e di bordo del dominio con campionamento uniforme
    anche dei parametri mu0, mu1 negli intervalli specificati
    """
    mu0_min, mu0_max = mu0_range
    mu1_min, mu1_max = mu1_range

    # --- Punti interni ---
    x0 = torch.rand(N_interior, 1)  # coordinate x0 in (0,1)
    x1 = torch.rand(N_interior, 1)  # coordinate x1 in (0,1)
    mu0 = mu0_min + (mu0_max - mu0_min) * torch.rand(N_interior, 1)  # parametri mu0 casuali nell’intervallo
    mu1 = mu1_min + (mu1_max - mu1_min) * torch.rand(N_interior, 1)  # parametri mu1 casuali nell’intervallo
    xmu_interior = torch.cat([x0, x1, mu0, mu1], dim=1)

    # --- Punti sul bordo ---
    xb = []
    for side in range(4):
        s = torch.rand(N_boundary, 1)          # coordinata variabile sul lato
        zeros = torch.zeros_like(s)             # vettore di zeri
        ones = torch.ones_like(s)               # vettore di uni

        if side == 0:
            # lato inferiore y=0
            x0b, x1b = s, zeros
        elif side == 1:
            # lato superiore y=1
            x0b, x1b = s, ones
        elif side == 2:
            # lato sinistro x=0
            x0b, x1b = zeros, s
        else:
            # lato destro x=1
            x0b, x1b = ones, s

        mu0b = mu0_min + (mu0_max - mu0_min) * torch.rand(N_boundary, 1)  # mu0 bordo
        mu1b = mu1_min + (mu1_max - mu1_min) * torch.rand(N_boundary, 1)  # mu1 bordo

        xb.append(torch.cat([x0b, x1b, mu0b, mu1b], dim=1))

    xmu_boundary = torch.cat(xb, dim=0)
    n_boundary_points = len(xmu_boundary)



    return xmu_interior, xmu_boundary

In [10]:

def train_pinn_poly(
        net, net_name,
        epochs=20000, N_interior=1500, N_boundary=200,
        lr=1e-3, mu0_range=(0.1, 1.0), mu1_range=(0.1, 1.0),
        power=2.0, seed=23,
        lambda_weight=1.0,               # <-- aggiunto parametro lambda di default 1.0
        csv_path=None,
        log_step=200
):
    # Se non specificato, crea il nome del file CSV includendo lambda_weight
    if csv_path is None:
        csv_path = f"training_log_lambda{lambda_weight:.3f}.csv"

    # reproducibility --------------------------------------------------
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net.to(device)

    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: (1 - epoch / epochs) ** power)

    # prepara CSV ------------------------------------------------------
    first_write = not os.path.exists(csv_path)
    with open(csv_path, "a", newline="") as csv_file:
        writer = csv.writer(csv_file)
        if first_write:
            writer.writerow(["net", "epoch",
                             "loss", "pde_loss", "bc_loss",
                             "epoch_time_s", "elapsed_s", "lambda_weight"])

    # training ---------------------------------------------------------
    t0 = time.time()
    prev_time = t0

    for epoch in range(1, epochs + 1):
        optimizer.zero_grad()
        net.train()

        # sample points
        # sono i punti che uso ad ogni epoca per addestrarre come se fosserro un unico grande batch
        # in questo caso 1500 punti interni e 200*4 punti sui bordi ad ogni epoca

        xmu_int, xmu_bnd = generate_domain_points(
            N_interior, N_boundary, mu0_range, mu1_range
        )
        xmu_int, xmu_bnd = xmu_int.to(device), xmu_bnd.to(device)

        # losses
        res_int  = pde_residual(xmu_int, net)
        loss_pde = torch.mean(res_int ** 2)
        u_bnd    = net(xmu_bnd)
        loss_bc  = torch.mean(u_bnd ** 2)
        loss     = loss_pde + lambda_weight * loss_bc    # usa lambda_weight qui

        # optimise
        loss.backward()
        optimizer.step()
        scheduler.step()

        # timing
        now = time.time()
        epoch_time = now - prev_time
        elapsed    = now - t0
        prev_time  = now


        if epoch % log_step == 0 or epoch == 1 or epoch == epochs:
            log = (f"[{net_name}] Ep {epoch}: "
                   f"loss={loss.item():.6f}, pde={loss_pde.item():.6f}, "
                   f"bc={loss_bc.item():.6f}, λ={lambda_weight:.3f}, "
                   f"dt={epoch_time:.3f}s, "
                   f"elapsed={elapsed/60:.1f}m")
            print(log)

            with open(csv_path, "a", newline="") as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow([net_name, epoch,
                                 f"{loss.item():.6f}",
                                 f"{loss_pde.item():.6f}",
                                 f"{loss_bc.item():.6f}",
                                 f"{epoch_time:.3f}",
                                 f"{elapsed:.3f}",
                                 f"{lambda_weight:.3f}"])

    total_min = (time.time() - t0) / 60
    print(f"{net_name} training complete in {total_min:.1f} min "
          f"(final loss {loss.item():.6f})")


**ORA ADDESTRO AL RETE CHE HO USATO NELL'ES1: NET11**

In [11]:

# Addestra tutte le reti e salva i risultati
lambda_values = [1.0]
net = Net11()
net_name = 'Net11()'
train_pinn_poly(
        net=net,
        net_name=net_name,
        lambda_weight=1.0)


[Net11()] Ep 1: loss=2400.819336, pde=2400.816162, bc=0.003235, λ=1.000, dt=0.825s, elapsed=0.0m
[Net11()] Ep 200: loss=44.433414, pde=36.974079, bc=7.459336, λ=1.000, dt=0.021s, elapsed=0.1m
[Net11()] Ep 400: loss=22.730743, pde=20.203077, bc=2.527665, λ=1.000, dt=0.019s, elapsed=0.1m
[Net11()] Ep 600: loss=5.245775, pde=3.550380, bc=1.695395, λ=1.000, dt=0.019s, elapsed=0.2m
[Net11()] Ep 800: loss=3.823333, pde=2.423090, bc=1.400243, λ=1.000, dt=0.019s, elapsed=0.3m
[Net11()] Ep 1000: loss=5.496253, pde=4.530099, bc=0.966154, λ=1.000, dt=0.019s, elapsed=0.4m
[Net11()] Ep 1200: loss=1.923845, pde=1.115642, bc=0.808203, λ=1.000, dt=0.019s, elapsed=0.4m
[Net11()] Ep 1400: loss=4.280752, pde=3.431996, bc=0.848756, λ=1.000, dt=0.019s, elapsed=0.5m
[Net11()] Ep 1600: loss=1.384222, pde=0.732410, bc=0.651812, λ=1.000, dt=0.019s, elapsed=0.6m
[Net11()] Ep 1800: loss=1.736830, pde=1.172456, bc=0.564374, λ=1.000, dt=0.020s, elapsed=0.6m
[Net11()] Ep 2000: loss=1.795762, pde=1.344800, bc=0.4509

In [12]:
quadr_coords = []
file_path = "/content/SolutionOnPoints_0.inp"

with open(file_path, "r") as f:
    next(f)  # salto header se serve
    for line in f:
        parts = line.strip().split()
        if len(parts) < 4:
            continue
        # Provo a convertire, se fallisce salto la riga
        try:
            idx = int(parts[0])
            x = float(parts[1])
            y = float(parts[2])
        except ValueError:
            # Righe non dati, le salto
            continue

        quadr_coords.append((x, y))
quadr_coords = np.array(quadr_coords)  # shape (N,2)
print("Coordinate shape:", quadr_coords.shape)
print("Prime 5 coordinate:", quadr_coords[:5])
print(quadr_coords.shape)


Coordinate shape: (4623, 2)
Prime 5 coordinate: [[0.9298771  0.46188024]
 [0.91123877 0.45266557]
 [0.93072317 0.44540916]
 [0.48698419 0.27382701]
 [0.4980866  0.26058181]]
(4623, 2)


In [13]:
# Choose new random seed bc we need a new test set
#np.random.seed(23) #gia fatto sopra

test_set = np.random.uniform(low=0.1, high=1, size=(100, 2)) # test set
test_set[-1]

array([0.84508049, 0.3486542 ])

In [14]:
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.eval() # gia sta in gpu dal train

coords_tensor = torch.tensor(quadr_coords, dtype=torch.float32)  # (N, 2)

all_preds = []  # lista per salvare le predizioni per ogni mu

with torch.no_grad():
    for mu in test_set:
        mu0_val, mu1_val = mu

        mu0_tensor = torch.full((coords_tensor.shape[0], 1), fill_value=mu0_val, dtype=torch.float32)
        mu1_tensor = torch.full((coords_tensor.shape[0], 1), fill_value=mu1_val, dtype=torch.float32)

        inputs = torch.cat([coords_tensor, mu0_tensor, mu1_tensor], dim=1)  # (N,4)
        inputs = inputs.to(device) #manda in gpu
        u_pred = net(inputs).cpu().numpy().flatten()  # (N,)

        all_preds.append(u_pred)

# Trasponi per avere righe = punti quadratura, colonne = diversi mu
all_preds = np.array(all_preds).T  # shape (N_punti, N_mu)

# Crea nomi colonne (es: mu0_0.5_mu1_0.5)
col_names = [f"mu0_{mu[0]:.3f}_mu1_{mu[1]:.3f}" for mu in test_set]

# Crea DataFrame pandas
df = pd.DataFrame(all_preds, columns=col_names)

# Salva su CSV
df.to_csv("predizioni_punti_quadratura_net11_es2.csv", index=False)

print("Salvataggio completato: predizioni_punti_quadratura_net11_es2.csv")


Salvataggio completato: predizioni_punti_quadratura_net11_es2.csv


In [15]:
last_mu = test_set[-1]
last_mu

array([0.84508049, 0.3486542 ])